In [1]:
import os
import time

import pandas as pd
import tensorflow as tf
from keras.callbacks import TensorBoard
from tensorflow.keras import backend as K
from tensorflow.keras.metrics import Precision, Recall, AUC
from tensorflow.keras.preprocessing.image import ImageDataGenerator

MODEL_NAME = "MegaClassifier_a"
VERSION_BASE = "v3"
EPOCHS = 10
BATCH_SIZE = 32
IMAGE_SIZE = (224, 224)
IMAGE_SHAPE = IMAGE_SIZE + (3,)
SEED = 42

DATASET_CSV = os.path.abspath("./data/processed/onlyDetectionsForTrain/onlyDetectionsForTrain.csv")
DATASET_PATH = os.path.dirname(DATASET_CSV)

dataset = pd.read_csv(DATASET_CSV, sep=";")
dataset['file_name'] = dataset['file_name'].apply(lambda x: os.path.join(DATASET_PATH, x))
dataset['binary_label'] = dataset['binary_label'].astype(str)

train_df = dataset[dataset['subset'] == 'train']
validation_df = dataset[dataset['subset'] == 'validation']
test_df = dataset[dataset['subset'] == 'test']


def focal_loss(alpha=0.25, gamma=2.0):
    def loss(y_true, y_pred):
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1.0 - epsilon)
        pt = tf.where(K.equal(y_true, 1), y_pred, 1 - y_pred)
        loss = -K.mean(alpha * K.pow(1. - pt, gamma) * K.log(pt))
        return loss

    return loss


def train_and_evaluate(alpha, gamma, VERSION_INDEX):
    loss_name = f"FocalLoss_alpha{alpha}_gamma{gamma}"
    print(f"\n🔹 Entrenando con Focal Loss (alpha={alpha}, gamma={gamma}) 🔹")

    train_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input)
    datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input)

    train_images = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        x_col="file_name",
        y_col="binary_label",
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode="binary",
        shuffle=True,
        seed=SEED,
    )
    validation_images = datagen.flow_from_dataframe(
        dataframe=validation_df,
        x_col="file_name",
        y_col="binary_label",
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode="binary",
        shuffle=True,
        seed=SEED,
    )
    test_images = datagen.flow_from_dataframe(
        dataframe=test_df,
        x_col="file_name",
        y_col="binary_label",
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode="binary",
    )

    mobilenet_v2 = tf.keras.applications.MobileNetV2(
        weights="imagenet",
        include_top=False,
        input_shape=IMAGE_SHAPE,
    )
    mobilenet_v2.trainable = False

    model = tf.keras.Sequential([
        mobilenet_v2,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(1, activation="sigmoid"),
    ], name=f"{MODEL_NAME}_{VERSION_BASE}.{VERSION_INDEX}")

    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=focal_loss(alpha=alpha, gamma=gamma),
        metrics=["accuracy", Precision(name="precision"), Recall(name="recall"), AUC(name="auc")],
    )

    start_time = time.time()
    history = model.fit(
        train_images,
        epochs=EPOCHS,
        validation_data=validation_images,
        callbacks=[
            TensorBoard(log_dir=f"./logs/{MODEL_NAME}/{VERSION_BASE}/{VERSION_BASE}.{VERSION_INDEX}"),
        ]
    )
    training_time = time.time() - start_time

    results = model.evaluate(test_images)

    history_df = pd.DataFrame(history.history)
    os.makedirs(f"./logs/{MODEL_NAME}/{VERSION_BASE}/{VERSION_BASE}.{VERSION_INDEX}", exist_ok=True)
    history_df.to_csv(
        f"./logs/{MODEL_NAME}/{VERSION_BASE}/{VERSION_BASE}.{VERSION_INDEX}/history_{VERSION_BASE}.{VERSION_INDEX}.csv",
        index=False)

    metric_names = history.model.metrics_names
    evaluation_results = {("test_" + name): value for name, value in zip(metric_names, results)}
    evaluation_results["alpha"] = alpha
    evaluation_results["gamma"] = gamma
    evaluation_results["training_time"] = training_time

    results_df = pd.DataFrame([evaluation_results])
    results_df.to_csv(
        f"./logs/{MODEL_NAME}/{VERSION_BASE}/{VERSION_BASE}.{VERSION_INDEX}/results_{VERSION_BASE}.{VERSION_INDEX}.csv",
        index=False)

    print(f"\n📉 Loss: {results[0]:.4f}")
    print(f"🎯 Accuracy: {results[1]:.4%}")
    print(f"✅ Precision: {results[2]:.4%}")
    print(f"🔄 Recall: {results[3]:.4%}")
    print(f"📊 AUC: {results[4]:.4f}")
    print(f"⏳ Tiempo de entrenamiento: {training_time:.2f} segundos")

    return results_df


alpha_values = [0.25, 0.5, 0.75]
gamma_values = [1.0, 2.0, 5.0]

count = 0
all_results = []
for alpha in alpha_values:
    for gamma in gamma_values:
        results_df = train_and_evaluate(alpha, gamma, count)
        all_results.append(results_df)
        count += 1

final_results = pd.concat(all_results, ignore_index=True)
final_results.to_csv(f"./logs/{MODEL_NAME}/{VERSION_BASE}/focal_loss_tuning_results.csv", index=False)

print("\n✅ ¡Experimento con optimización de Focal Loss completado!")


🔹 Entrenando con Focal Loss (alpha=0.25, gamma=1.0) 🔹
Found 17054 validated image filenames belonging to 2 classes.
Found 4286 validated image filenames belonging to 2 classes.
Found 4286 validated image filenames belonging to 2 classes.


2025-02-23 19:43:26.456411: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3
2025-02-23 19:43:26.456438: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-02-23 19:43:26.456442: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-02-23 19:43:26.456598: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-02-23 19:43:26.456610: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/10


2025-02-23 19:43:28.053992: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


533/533 [==============================] - ETA: 0s - loss: 0.0346 - accuracy: 0.8813 - precision: 0.9110 - recall: 0.9387 - auc: 0.9324

...

Epoch 1/10


2025-02-23 20:57:30.891801: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


533/533 [==============================] - ETA: 0s - loss: 0.0107 - accuracy: 0.8629 - precision: 0.9126 - recall: 0.9104 - auc: 0.9174

2025-02-23 20:58:14.175070: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


533/533 [==============================] - 55s 101ms/step - loss: 0.0107 - accuracy: 0.8629 - precision: 0.9126 - recall: 0.9104 - auc: 0.9174 - val_loss: 0.0068 - val_accuracy: 0.9078 - val_precision: 0.9427 - val_recall: 0.9164 - val_auc: 0.9658
Epoch 2/10
533/533 [==============================] - 54s 101ms/step - loss: 0.0063 - accuracy: 0.9024 - precision: 0.9347 - recall: 0.9397 - auc: 0.9574 - val_loss: 0.0057 - val_accuracy: 0.9258 - val_precision: 0.9321 - val_recall: 0.9577 - val_auc: 0.9762
Epoch 3/10
533/533 [==============================] - 53s 99ms/step - loss: 0.0057 - accuracy: 0.9100 - precision: 0.9403 - recall: 0.9438 - auc: 0.9642 - val_loss: 0.0064 - val_accuracy: 0.9274 - val_precision: 0.9261 - val_recall: 0.9676 - val_auc: 0.9769
Epoch 4/10
533/533 [==============================] - 53s 99ms/step - loss: 0.0049 - accuracy: 0.9227 - precision: 0.9479 - recall: 0.9527 - auc: 0.9719 - val_loss: 0.0058 - val_accuracy: 0.9186 - val_precision: 0.9687 - val_recall: 0.